# Advent of Code 2022

## Day 21: Monkey Math

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Day 21 was fun and not too hard. The monkeys are nodes in a tree, with number-only monkeys as leafs and calculating monkeys as intermediate nodes. Actually the data could be a DAG not a tree but I didn't check for that as it didn't really matter. In part 1 I just make `ValueMonkey` for the leaves, and `OpMonkey` for the intermediate nodes. A compile step wires their pointers togethor, and a call to `query()` on `root` recursively calls `query()` on child monkeys. I used `Fraction` instead of `int` in case there were non-integer divisions.

Part 2 I used `sympy` because just iterating over possible inputs was taking too long even with caching parts of the tree which don't depend on input. So now `Monkey.symbolic()` returns a value sympy can solve using similar logic to `query()` but passing a big `Eq` object to be solved. `HumanPretendingToBeMonkey` returns a `symbols('humn')` which I solved for.

### Imports

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional, Union
from fractions import Fraction
from sympy import symbols, Eq, solve

### Monkeys Objects

In [ ]:
class Monkey(ABC):

    def __init__(self, name: str):
        self.name = name

    @abstractmethod
    def compile(self, monkeys: dict[str, 'Monkey']) -> None:
        pass

    @abstractmethod
    def query(self) -> Fraction:
        pass

    @abstractmethod
    def symbolic(self):
        pass


class ValueMonkey(Monkey):

    def __init__(self, name, value: int):
        super().__init__(name)
        self.value = value

    def compile(self, monkeys: dict[str, Monkey]) -> None:
        pass

    def query(self) -> Fraction:
        return Fraction(self.value)

    def symbolic(self):
        return self.value


class OpMonkey(Monkey):

    def __init__(self, name: str, left: str, op: str, right: str):
        super().__init__(name)
        self.left: Union[str, Monkey] = left
        self.op: str = op
        self.right: Union[str, Monkey] = right
        self.value: Optional[Fraction] = None

    def compile(self, monkeys: dict[str, Monkey]) -> None:
        assert isinstance(self.left, str) and isinstance(self.right, str)
        self.left = monkeys[self.left]
        self.right = monkeys[self.right]

    def query(self) -> Fraction:
        if self.op == '=':
            raise AssertionError(f'invalid op: = with method query(), use symbolic()')
        elif self.op == '+':
            return self.left.query() + self.right.query()
        elif self.op == '-':
            return self.left.query() - self.right.query()
        elif self.op == '*':
            return self.left.query() * self.right.query()
        elif self.op == '/':
            return self.left.query() / self.right.query()
        else:
            raise AssertionError(f'invalid op: {self.op}')

    def symbolic(self):
        if self.op == '=':
            return Eq(self.left.symbolic(), self.right.symbolic())
        elif self.op == '+':
            return self.left.symbolic() + self.right.symbolic()
        elif self.op == '-':
            return self.left.symbolic() - self.right.symbolic()
        elif self.op == '*':
            return self.left.symbolic() * self.right.symbolic()
        elif self.op == '/':
            return self.left.symbolic() / self.right.symbolic()
        else:
            raise AssertionError(f'invalid op: {self.op}')



class HumanPretendingToBeMonkey(Monkey):

    def __init__(self, name: str):
        super().__init__(name)
        self.symbol = symbols('humn')

    def compile(self, monkeys: dict[str, Monkey]) -> None:
        pass

    def query(self) -> Fraction:
        raise AssertionError(f'HumanPretendingToBeMonkey can not participate in query(), use symbolic()')

    def symbolic(self):
        return self.symbol

### Input Parsing

In [ ]:
def make_monkeys(filenaname: str) -> dict[str, Monkey]:
    monkeys = {}
    with open(filenaname) as file:
        for line in file:
            line = line.strip()
            monkey, inst_raw = line.split(': ')
            if ' ' in inst_raw:
                l, o, r = inst_raw.split(' ')
                monkeys[monkey] = OpMonkey(monkey, l, o, r)
            else:
                monkeys[monkey] = ValueMonkey(monkey, int(inst_raw))
    return monkeys

In [ ]:
def compile_all(monkeys: dict[str, Monkey]) -> None:
    for m in monkeys.values():
        m.compile(monkeys)

In [ ]:
INPUT_FILE = 'data/input21.txt'

### Part 1

In [ ]:
def main():

    # make monkey
    monkeys = make_monkeys(INPUT_FILE)

    # compile monkey
    compile_all(monkeys)

    # get monkey
    root = monkeys['root']

    # query monkey
    ans = root.query()

    print(f'The monkey named root will yell the number {int(ans)}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def main():

    # make monkey
    monkeys = make_monkeys(INPUT_FILE)

    # become monkey
    human = HumanPretendingToBeMonkey('humn')
    monkeys['humn'] = human

    # get monkey
    root = monkeys['root']

    # update monkey
    root.op = '='

    # compile monkey
    compile_all(monkeys)

    # solve monkey
    ans = int(solve(root.symbolic())[0])

    print(f'I should yell the number {int(ans)}')

In [ ]:
if __name__ == '__main__':
    main()